# Delhi Colonies Public Services Index (25 August 2020)

## Compute the following indices:
* Index with touching neighbors [effective service count divided by population]
* Index with bounding box neighbors [effective service count divided by population]
* Index with touching neighbors [effective service count divided by population/area]
* Index with bounding box neighbors [effective service count divided by population/area]


### How to compute indices
* Load in colonies datasets from Pickle **[done]**
* Import services shapefiles **[done]**
    * Make sure correct file paths exist
    * Ensure that all shapefiles are valid using `check_shapefile` function
    * Reproject shapefiles to EPSG 7760 (if needed)
* Compute all Services indices (turn into a function)
    * Touching neighbors, Population Size
    * Touching neighbors, Population Density
    * bbox neighbors, Population Size
    * bbox neighbors, Population Density

## Import modules and set constants

In [2]:
import os
import pickle
from importlib import reload
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, box
import spatial_index_utils

In [3]:
reload(spatial_index_utils)

<module 'spatial_index_utils' from 'C:\\Users\\bwbel\\Google Drive\\slum_project\\spatial_index_python\\spatial_index_utils.py'>

In [4]:
# WGS 84 / Delhi
epsg_code = 7760

## Import Colonies Datasets

In [5]:
with open('colonies_touch_nbrs25Aug2020.pkl', 'rb') as f:
    colonies_touch_nbrs = pickle.load(f)
    
colonies_touch_nbrs.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,area_km2,canal,railway,drain,barrier,centroid,ndmc_dist_km,population,nbrs_touch,nbrs_dist_touch,index
0,NEW DELHI 36,5584,None,Planned,"POLYGON Z ((1020282.788 996796.773 0.000, 1020...",1.966739,False,True,False,True,POINT (1020123.175 995898.851),5.159809,3570.060984,"[5598, 5599, 5602, 5603, 5605, 3491, 3508, 377...","[(5598, 1.074790368771482), (5599, 1.015506410...",0
1,NEW DELHI 35,5585,None,Planned,"POLYGON Z ((1019724.475 994932.797 0.000, 1019...",0.036429,False,False,False,False,POINT (1019673.024 994869.699),6.273149,323.028887,"[5586, 5594]","[(5586, 0.35712228070675794), (5594, 0.6299162...",1
2,NEW DELHI 34,5586,None,Planned,"POLYGON Z ((1019571.955 994876.019 0.000, 1019...",0.230739,False,False,False,False,POINT (1019485.484 994565.783),6.618792,2215.206473,"[5585, 5587, 5594, 5596]","[(5585, 0.35712228070675794), (5587, 0.3138054...",2
3,NEW DELHI 33,5587,None,Planned,"POLYGON Z ((1019352.702 994352.546 0.000, 1019...",0.281195,False,False,False,False,POINT (1019171.868 994576.688),6.709542,3956.166944,"[5586, 5588, 5596]","[(5586, 0.3138054943365384), (5588, 0.38643264...",3
4,NEW DELHI 32,5588,None,Planned,"POLYGON Z ((1018793.292 994224.182 0.000, 1018...",0.301253,False,False,False,False,POINT (1018785.675 994590.275),6.839299,3961.943378,"[5587, 5596, 5620, 5621]","[(5587, 0.3864326417184364), (5596, 0.62727257...",4


In [6]:
with open('colonies_bbox_nbrs25Aug2020.pkl', 'rb') as f:
    colonies_bbox_nbrs = pickle.load(f)
    
colonies_bbox_nbrs.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,area_km2,canal,railway,drain,barrier,centroid,ndmc_dist_km,population,nbrs_bbox,nbrs_dist_bbox,index
0,NEW DELHI 36,5584,None,Planned,"POLYGON Z ((1020282.788 996796.773 0.000, 1020...",1.966739,False,True,False,True,POINT (1020123.175 995898.851),5.159809,3570.060984,"[5598, 5599, 5602, 5603, 5605, 3491, 3508, 377...","[(5598, 1.074790368771482), (5599, 1.015506410...",0
1,NEW DELHI 35,5585,None,Planned,"POLYGON Z ((1019724.475 994932.797 0.000, 1019...",0.036429,False,False,False,False,POINT (1019673.024 994869.699),6.273149,323.028887,"[5586, 5594]","[(5586, 0.35712228070675794), (5594, 0.6299162...",1
2,NEW DELHI 34,5586,None,Planned,"POLYGON Z ((1019571.955 994876.019 0.000, 1019...",0.230739,False,False,False,False,POINT (1019485.484 994565.783),6.618792,2215.206473,"[5585, 5587, 5594, 5596]","[(5585, 0.35712228070675794), (5587, 0.3138054...",2
3,NEW DELHI 33,5587,None,Planned,"POLYGON Z ((1019352.702 994352.546 0.000, 1019...",0.281195,False,False,False,False,POINT (1019171.868 994576.688),6.709542,3956.166944,"[5586, 5588, 5596]","[(5586, 0.3138054943365384), (5588, 0.38643264...",3
4,NEW DELHI 32,5588,None,Planned,"POLYGON Z ((1018793.292 994224.182 0.000, 1018...",0.301253,False,False,False,False,POINT (1018785.675 994590.275),6.839299,3961.943378,"[5587, 5596, 5620, 5621]","[(5587, 0.3864326417184364), (5596, 0.62727257...",4


## Import services shapefiles

In [7]:
# Define filepaths

services_dir = os.path.join('shapefiles', 'Spatial_Index_GIS', 'Public Services')

bank_fp = os.path.join(services_dir, 'Banking', 'Banking.shp')
health_fp = os.path.join(services_dir, 'Health', 'Health.shp')
road_fp = os.path.join(services_dir, 'Major Road', 'Road.shp')
police_fp = os.path.join(services_dir, 'Police', 'Police Station.shp')
ration_fp = os.path.join(services_dir, 'Ration', 'Ration.shp')
school_fp = os.path.join(services_dir, 'School', 'schools7760.shp')
transport_fp = os.path.join(services_dir, 'Transport', 'Transport.shp')

# boundary of Delhi
delhi_bounds_filepath = os.path.join('shapefiles', 'delhi_bounds_buffer.shp')

# Check that all filepaths exist
filepath_list = [bank_fp, health_fp, road_fp, police_fp, ration_fp, school_fp, transport_fp, delhi_bounds_filepath]

for filepath in filepath_list:
    if not os.path.exists(filepath):
        print('{} does not exist'.format(filepath))

In [8]:
# Import services
bank = gpd.read_file(bank_fp)
health = gpd.read_file(health_fp)
road = gpd.read_file(road_fp)
police = gpd.read_file(police_fp)
ration = gpd.read_file(ration_fp)
school = gpd.read_file(school_fp)
transport = gpd.read_file(transport_fp)

## Check validity of services shapefiles
* Duplicate rows are okay for ATMs (I assume that ATM locations for the same bank in a similar location will seem to be counted twice)
* Look specifically for invalid geometries and whether shapefile is fully contained within Delhi

In [9]:
spatial_index_utils.check_shapefile(gdf=bank, gdf_name='bank', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

bank has duplicate rows: True
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in bank are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [bank_name, Latitude, Longitude, Type, geometry, geom_type]
Index: []
----------------------------------------------------
bank shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [10]:
spatial_index_utils.check_shapefile(gdf=health, gdf_name='health', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

health has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in health are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [Hospital_C, ADDRESS, X, Y, geometry, geom_type]
Index: []
----------------------------------------------------
health shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [11]:
spatial_index_utils.check_shapefile(gdf=road, gdf_name='road', 
                                    geom_type='Line', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

road has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in road are of type Line: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [FID, RD_NM, RD_CLS, RD_LANES, RD_TP_SRF, RD_MB, RD_ONEWAY, EL_GND, DIST_NM, ONEWAY, Speed_kmph, geometry, geom_type]
Index: []
----------------------------------------------------
road shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [12]:
spatial_index_utils.check_shapefile(gdf=police, gdf_name='police', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

police has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in police are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [NAME, POLICE_STA, DISTRICT, x, y, geometry, geom_type]
Index: []
----------------------------------------------------
police shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [13]:
spatial_index_utils.check_shapefile(gdf=ration, gdf_name='ration', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

ration has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in ration are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [S No., License No, FPS ID, Circle, FPS Shop N, Address Of, Latitude, Longitude, Source, geometry, geom_type]
Index: []
----------------------------------------------------
ration shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [14]:
spatial_index_utils.check_shapefile(gdf=school, gdf_name='school', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

school has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in school are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [objectid_1, objectid, vilname, schname, schcd, schcat, school_cat, pincode, rururb, location, schtype, school_typ, schmgt, management, dtname, stname, stcode11, dtcode11, sdtcode11, sdtname, geometry, geom_type]
Index: []

[0 rows x 22 columns]
----------------------------------------------------
school shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [15]:
spatial_index_utils.check_shapefile(gdf=transport, gdf_name='transport', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

transport has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in transport are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [stop_id, stop_name, stop_lat, stop_lon, Type, geometry, geom_type]
Index: []
----------------------------------------------------
transport shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


## Check CRS (all shapefiles should be in EPSG: 7760)

In [16]:
bank.crs == health.crs == road.crs == police.crs == ration.crs == school.crs == transport.crs

True

In [17]:
bank.crs

<Projected CRS: EPSG:7760>
Name: WGS 84 / Delhi
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: India - Delhi
- bounds: (76.83, 28.4, 77.34, 28.89)
Coordinate Operation:
- name: Delhi NSF LCC
- method: Lambert Conic Conformal (2SP)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [19]:
colonies_bbox_nbrs.crs == colonies_touch_nbrs.crs == bank.crs

True

## Define Point and Line Services

In [20]:
# Define all point services as dictionary
# makes it easier to calculate all point
# services with one function
point_services = {'bank': bank,
                  'health': health,
                  'police': police,
                  'ration': ration,
                  'school': school,
                  'transport': transport}

line_services = {'road': road}

## Calculate all service indices in one function

In [21]:
from spatial_index_utils import calc_all_services

In [22]:
calc_all_services?

Signature:
calc_all_services(
    polygon_gdf,
    point_services,
    line_services,
    epsg_code,
    calc_pop_density,
    nbr_dist_colname,
)
Docstring: Calculate all public services indices (point and line)
File:      c:\users\bwbel\google drive\slum_project\spatial_index_python\spatial_index_utils.py
Type:      function


### Calculate PSI for touching neighbors using Population Size

In [23]:
colonies_touch_psi_popsize = calc_all_services(polygon_gdf = colonies_touch_nbrs, 
                                       point_services = point_services, 
                                       line_services = line_services, 
                                       epsg_code = epsg_code, 
                                       calc_pop_density = False,
                                       nbr_dist_colname = 'nbrs_dist_touch')

colonies_touch_psi_popsize = colonies_touch_psi_popsize.rename(columns={'road_count':'road_length'})
colonies_touch_psi_popsize

GeoDataFrame now has the following CRS:

epsg:7760
bank service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
health service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
police service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
ration service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
school service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
transport service index is completed
--------------------------------------------------------
all point services completed
GeoDataFrame now has the following CRS:

epsg:7760
road service is completed


,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,area_km2,canal,railway,drain,barrier,...,ration_idx,school_count,school_pcen,school_idx,transport_count,transport_pcen,transport_idx,road_length,road_pcen,road_idx
0,NEW DELHI 36,5584,None,Planned,"POLYGON Z ((1020282.788 996796.773 0.000, 1020...",1.966739,False,True,False,True,...,0.001943,0,0.000719,0.003438,3,0.006834,0.010744,1.951928,0.003171,0.007955
1,NEW DELHI 35,5585,None,Planned,"POLYGON Z ((1019724.475 994932.797 0.000, 1019...",0.036429,False,False,False,False,...,0.072328,0,0.002281,0.010908,0,0.015586,0.024503,0.000000,0.001117,0.002804
2,NEW DELHI 34,5586,None,Planned,"POLYGON Z ((1019571.955 994876.019 0.000, 1019...",0.230739,False,False,False,False,...,0.009533,1,0.003051,0.014587,6,0.004094,0.006437,0.000000,0.000681,0.001708
3,NEW DELHI 33,5587,None,Planned,"POLYGON Z ((1019352.702 994352.546 0.000, 1019...",0.281195,False,False,False,False,...,0.001642,6,0.002216,0.010597,0,0.002169,0.003409,0.000000,0.000437,0.001097
4,NEW DELHI 32,5588,None,Planned,"POLYGON Z ((1018793.292 994224.182 0.000, 1018...",0.301253,False,False,False,False,...,0.001366,1,0.002003,0.009577,2,0.001473,0.002316,0.747875,0.000650,0.001630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4347,"HARIJAN BASTI, SADAT PUR, DELHI-94.",5579,None,UAC,"POLYGON Z ((1028449.945 1004222.511 0.000, 102...",0.062420,False,False,False,False,...,0.004376,0,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000
4348,"CHRISTIAN COLONY, PATELCHEST, DELHI-7.",5580,None,UAC,"POLYGON Z ((1004320.138 1007798.133 0.000, 100...",0.031177,False,False,False,False,...,0.003527,0,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000
4349,"SULTANPUR MAZRA EXTN,. (EAST), NANGLOI, DELHI-41.",5581,None,UAC,"POLYGON Z ((1004443.956 1007676.189 0.000, 100...",0.015714,False,False,False,False,...,0.008620,0,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000
4350,"NAI BASTI BAKNER PASCHIM,DELHI.",5582,None,UAC,"POLYGON Z ((1003152.614 1011361.143 0.000, 100...",0.141523,False,False,False,False,...,0.019525,0,0.001144,0.005468,1,0.004188,0.006584,0.000000,0.000925,0.002321


### Calculate PSI for touching neighbors using Population Density

In [24]:
colonies_touch_psi_popdensity = calc_all_services(polygon_gdf = colonies_touch_nbrs, 
                                       point_services = point_services, 
                                       line_services = line_services, 
                                       epsg_code = epsg_code, 
                                       calc_pop_density = True,
                                       nbr_dist_colname = 'nbrs_dist_touch')

colonies_touch_psi_popdensity = colonies_touch_psi_popdensity.rename(columns={'road_count':'road_length'})
colonies_touch_psi_popdensity.head()

GeoDataFrame now has the following CRS:

epsg:7760
bank service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
health service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
police service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
ration service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
school service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
transport service index is completed
--------------------------------------------------------
all point services completed
GeoDataFrame now has the following CRS:

epsg:7760
road service is completed


,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,area_km2,canal,railway,drain,barrier,...,ration_idx,school_count,school_pcen,school_idx,transport_count,transport_pcen,transport_idx,road_length,road_pcen,road_idx
0,NEW DELHI 36,5584,None,Planned,"POLYGON Z ((1020282.788 996796.773 0.000, 1020...",1.966739,False,True,False,True,...,0.099680,0,0.001414,0.132334,3,0.013441,0.411243,1.951928,0.006236,0.280386
1,NEW DELHI 35,5585,None,Planned,"POLYGON Z ((1019724.475 994932.797 0.000, 1019...",0.036429,False,False,False,False,...,0.068726,0,0.000083,0.007778,0,0.000568,0.017372,0.000000,0.000041,0.001830
2,NEW DELHI 34,5586,None,Planned,"POLYGON Z ((1019571.955 994876.019 0.000, 1019...",0.230739,False,False,False,False,...,0.057374,1,0.000704,0.065882,6,0.000945,0.028906,0.000000,0.000157,0.007064
3,NEW DELHI 33,5587,None,Planned,"POLYGON Z ((1019352.702 994352.546 0.000, 1019...",0.281195,False,False,False,False,...,0.012047,6,0.000623,0.058325,0,0.000610,0.018658,0.000000,0.000123,0.005529
4,NEW DELHI 32,5588,None,Planned,"POLYGON Z ((1018793.292 994224.182 0.000, 1018...",0.301253,False,False,False,False,...,0.010730,1,0.000603,0.056470,2,0.000444,0.013578,0.747875,0.000196,0.008799


### Calculate PSI for bbox neighbors using Population Size

In [25]:
colonies_bbox_psi_popsize = calc_all_services(polygon_gdf = colonies_bbox_nbrs, 
                                       point_services = point_services, 
                                       line_services = line_services, 
                                       epsg_code = epsg_code, 
                                       calc_pop_density = False,
                                       nbr_dist_colname = 'nbrs_dist_bbox')

colonies_bbox_psi_popsize = colonies_bbox_psi_popsize.rename(columns={'road_count':'road_length'})
colonies_bbox_psi_popsize.head()

GeoDataFrame now has the following CRS:

epsg:7760
bank service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
health service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
police service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
ration service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
school service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
transport service index is completed
--------------------------------------------------------
all point services completed
GeoDataFrame now has the following CRS:

epsg:7760
road service is completed


,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,area_km2,canal,railway,drain,barrier,...,ration_idx,school_count,school_pcen,school_idx,transport_count,transport_pcen,transport_idx,road_length,road_pcen,road_idx
0,NEW DELHI 36,5584,None,Planned,"POLYGON Z ((1020282.788 996796.773 0.000, 1020...",1.966739,False,True,False,True,...,0.001934,0,0.000719,0.002675,3,0.006834,0.004113,1.951928,0.003171,0.004338
1,NEW DELHI 35,5585,None,Planned,"POLYGON Z ((1019724.475 994932.797 0.000, 1019...",0.036429,False,False,False,False,...,0.072000,0,0.002281,0.008488,0,0.015586,0.009381,0.000000,0.001117,0.001529
2,NEW DELHI 34,5586,None,Planned,"POLYGON Z ((1019571.955 994876.019 0.000, 1019...",0.230739,False,False,False,False,...,0.009490,1,0.003051,0.011351,6,0.004094,0.002464,0.000000,0.000681,0.000932
3,NEW DELHI 33,5587,None,Planned,"POLYGON Z ((1019352.702 994352.546 0.000, 1019...",0.281195,False,False,False,False,...,0.001635,6,0.002216,0.008246,0,0.002169,0.001305,0.000000,0.000437,0.000598
4,NEW DELHI 32,5588,None,Planned,"POLYGON Z ((1018793.292 994224.182 0.000, 1018...",0.301253,False,False,False,False,...,0.001359,1,0.002003,0.007452,2,0.001473,0.000887,0.747875,0.000650,0.000889


### Calculate PSI for touching neighbors using Population Density

In [26]:
colonies_bbox_psi_popdensity = calc_all_services(polygon_gdf = colonies_bbox_nbrs, 
                                       point_services = point_services, 
                                       line_services = line_services, 
                                       epsg_code = epsg_code, 
                                       calc_pop_density = True,
                                       nbr_dist_colname = 'nbrs_dist_bbox')

colonies_bbox_psi_popdensity = colonies_bbox_psi_popdensity.rename(columns={'road_count':'road_length'})
colonies_bbox_psi_popdensity.head()

GeoDataFrame now has the following CRS:

epsg:7760
bank service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
health service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
police service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
ration service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
school service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
transport service index is completed
--------------------------------------------------------
all point services completed
GeoDataFrame now has the following CRS:

epsg:7760
road service is completed


,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,area_km2,canal,railway,drain,barrier,...,ration_idx,school_count,school_pcen,school_idx,transport_count,transport_pcen,transport_idx,road_length,road_pcen,road_idx
0,NEW DELHI 36,5584,None,Planned,"POLYGON Z ((1020282.788 996796.773 0.000, 1020...",1.966739,False,True,False,True,...,0.099680,0,0.001414,0.132334,3,0.013441,0.411243,1.951928,0.006236,0.280386
1,NEW DELHI 35,5585,None,Planned,"POLYGON Z ((1019724.475 994932.797 0.000, 1019...",0.036429,False,False,False,False,...,0.068726,0,0.000083,0.007778,0,0.000568,0.017372,0.000000,0.000041,0.001830
2,NEW DELHI 34,5586,None,Planned,"POLYGON Z ((1019571.955 994876.019 0.000, 1019...",0.230739,False,False,False,False,...,0.057374,1,0.000704,0.065882,6,0.000945,0.028906,0.000000,0.000157,0.007064
3,NEW DELHI 33,5587,None,Planned,"POLYGON Z ((1019352.702 994352.546 0.000, 1019...",0.281195,False,False,False,False,...,0.012047,6,0.000623,0.058325,0,0.000610,0.018658,0.000000,0.000123,0.005529
4,NEW DELHI 32,5588,None,Planned,"POLYGON Z ((1018793.292 994224.182 0.000, 1018...",0.301253,False,False,False,False,...,0.010730,1,0.000603,0.056470,2,0.000444,0.013578,0.747875,0.000196,0.008799


## Save Files

In [27]:
bbox_drop_columns = ['nbrs_bbox', 'nbrs_dist_bbox', 'centroid']
touch_drop_columns = ['nbrs_touch', 'nbrs_dist_touch', 'centroid']

In [28]:
colonies_bbox_psi_popsize.drop(columns=bbox_drop_columns).to_file('delhi_psi_bbox_popsize_25Aug2020.shp')
colonies_bbox_psi_popdensity.drop(columns=bbox_drop_columns).to_file('delhi_psi_bbox_popdensity_25Aug2020.shp')

In [29]:
colonies_touch_psi_popsize.drop(columns=touch_drop_columns).to_file('delhi_psi_touch_popsize_25Aug2020.shp')
colonies_touch_psi_popdensity.drop(columns=touch_drop_columns).to_file('delhi_psi_touch_popdensity_25Aug2020.shp')

In [30]:
colonies_bbox_psi_popsize.to_csv('delhi_psi_bbox_popsize_25Aug2020.csv')
colonies_bbox_psi_popdensity.to_csv('delhi_psi_bbox_popdensity_25Aug2020.csv')

In [31]:
colonies_touch_psi_popsize.to_csv('delhi_psi_touch_popsize_25Aug2020.csv')
colonies_touch_psi_popdensity.to_csv('delhi_psi_touch_popdensity_25Aug2020.csv')

In [32]:
with open('colonies_bbox_psi_popsize.pkl', 'wb') as f:
    pickle.dump(colonies_bbox_psi_popsize, f)
    
with open('colonies_bbox_psi_popdensity.pkl', 'wb') as f:
    pickle.dump(colonies_bbox_psi_popdensity, f)

with open('colonies_touch_psi_popsize.pkl', 'wb') as f:
    pickle.dump(colonies_touch_psi_popsize, f)

with open('colonies_touch_psi_popdensity.pkl', 'wb') as f:
    pickle.dump(colonies_touch_psi_popdensity, f)